# Exercicio

Vamos usar um famoso dataset sobre preços de casas em Boston (década de 70) para analisar alguns pontos.

O conjunto de dados contém informações relacionadas à habitação na área de Boston, Massachusetts. Isso inclui recursos relacionados à estrutura, vizinhança, acessibilidade e qualidade do ar. Grande parte dos dados foi coletada pelo Serviço de Recenseamento dos EUA em 1970, mas o conjunto completo de dados foi reunido para a publicação original: Harrison, D. e Rubinfeld, preços DL 'Hedonic e a demanda por ar limpo', J. Environ. Economics & Management, volume 5, 81-102, 1978.

Cada observação na nossa amostra é um setor censitario que tem entre 1000 e 8000 pessoas.

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

%matplotlib inline
boston = load_boston()

In [ ]:
print(boston.keys())

In [ ]:
print(boston.data.shape)

In [ ]:
print(boston.feature_names)

In [ ]:
print(boston.DESCR)

In [ ]:
bos = pd.DataFrame(boston.data)

In [ ]:
bos.head()

# Guia

(Aqui começamos a analisar dados! Portanto não há uma resposta certa ou errada, há apenas boas práticas, então abuse de pesquisar formas diferentes de se fazer a mesma atividade).

### 1. Preencha os nomes das colunas do dataset.

In [ ]:
bos.columns = boston.feature_names

In [ ]:
bos.head()

### 2. Crie uma coluna chamada "MEDV" com o "target".

In [ ]:
bos['nova_variavel'] = boston.target
bos.head()

### 3. Análise as estatisticas descritivas de todas as variaveis numéricas. Há  campos nulos?

In [ ]:
# há muitas formas de se fazer isso, comece com o dataframe.describe() e analise os números.

In [ ]:
bos.describe().round(1)

### 4. Exiba a matriz de variância x covariância das variaveis. Quais são as 3 variaveis mais correlacionadas com o preço dos imóveis? E as três menos correlacionadas? (tente não buscar no visual .. )

In [ ]:
# a forma nao visual é essa: dataframe.corr().round(2) mas há muitos comandos mais intuitivos de viz.

In [ ]:
bos.corr().round(2)

In [ ]:
sns.pairplot(
    data=bos,
    vars=list(bos),
    size=1,
    plot_kws={'s': 5}
);

In [ ]:
corr = bos.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

with sns.axes_style("white"):
    fig, ax = plt.subplots(figsize=(15, 12));
    
    sns.heatmap(
        corr,
        ax=ax,
        annot=True,
        mask=mask,
        square=True
    );

### 5. Plot o histograma dos preços médios das casas e o histograma dos LOG dos preços médios das casas. A variável se parece mais com uma distribuição normal? (pode ser no visual).

In [ ]:
bos[['nova_variavel']].plot(
    kind='hist',
    figsize=(15, 5),
    bins=10
);

In [ ]:
np.log(bos[['nova_variavel']]).plot(
    kind='hist',
    figsize=(15, 5),
    bins=10
);

### 6. Rode uma regressão simples entre a preços médios do condado e as Idades médias desses imóveis.

    a) Qual o valor do nosso beta e o que ele significa?
    b) Esse parâmetro estimado é estatisticamente significativo? Qual a probabilidade dele ser diferente de zero?
    c) Podemos dizer que o o beta estimado (para idade) representa um efeito causal nos preços médios? Explique.
    d) Agora rode a mesma regressão com log no preço médio e interprete a diferença dos resultados (betas e R^2) porque se alterou?

In [ ]:
def run_regression(df,function):
    model = smf.ols(function, df).fit()
    print(model.summary())

In [ ]:
run_regression(bos,'''nova_variavel ~ AGE''')

In [ ]:
run_regression(bos,'''np.log(nova_variavel) ~ AGE''')

### 7. Quanto morar próximo ao Rio Charles impacta nos preços médios dos imóveis?

In [ ]:
bos.CHAS.value_counts()

In [ ]:
run_regression(bos,'''nova_variavel ~ CHAS''')

### 8. Rode agora uma nova regressão com todas as variáveis disponiveis.

    a) Porque o beta estimado para idade média se alterou? Análise os porquês dele ter perdido significância estatística.
    b) Explique porque o beta estimado para o Rio Charles se alterou.
    c) Qual o impacto da poluição nos preços dos imóveis?
    d) Podemos afirmar que os efeitos estimados são causais? Porque?
    e) Tem algum sinal dos parâmetros que não faz sentido? Explique.
    f) Qual a interpretação do intercepto nessa regressão.

In [ ]:
run_regression(bos,'''nova_variavel ~ CRIM +
ZN +
INDUS +
CHAS +
NOX +
RM +
AGE +
DIS +
RAD +
TAX +
PTRATIO +
B +
LSTAT''')

# Desafio

1) Rode uma regressão em que seus parâmetros estimados expliquem mais do que 80% da variância dos preços médios. 

In [ ]:
run_regression(bos,'''nova_variavel ~ CRIM +
ZN +
CHAS +
NOX +
RM +
DIS +
RAD +
TAX +
PTRATIO +
B +
LSTAT''')

In [ ]:
bos['CHAS_2'] = bos['CHAS']**2
bos['RM_2'] = bos['RM']**2

In [ ]:
run_regression(bos,'''nova_variavel ~ CRIM +
ZN +
CHAS +
CHAS_2 +
NOX +
RM +
RM_2 +
DIS +
RAD +
TAX +
PTRATIO +
B +
LSTAT''')

2) Se você fosse um negociante de imóveis e tivesse que comprar 5 imóveis visando a melhor rentabilidade percentual (retorno sobre o investimento), quais 5 imóveis do nosso dataset você apostaria? Porque? (dica do professor: Residuos) 

- Update: Considere que você deve escolher um condado (que há milhares de imóveis) ao invés de um imóvel já que o agrupamento dos dados é por condado.

In [ ]:
bos.shape

In [ ]:
function = '''nova_variavel ~ CRIM +
ZN +
CHAS +
CHAS_2 +
NOX +
RM +
RM_2 +
DIS +
RAD +
TAX +
PTRATIO +
B +
LSTAT'''

In [ ]:
model = smf.ols(function, bos).fit()
yhat = model.predict()

In [ ]:
residuo = yhat - bos.nova_variavel

In [ ]:
residuo.sort_values()[:5]